In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import numpy as np


In [ ]:
demographics = pd.read_csv("../data/processed/abs_demographics_merged.csv")
gdf_sa2 = gpd.read_file("../data/external/SA2_2021_AUST_GDA2020.shp")
gdf_sa2 = gdf_sa2[['SA2_CODE21', 'geometry']]
gdf_sa2.columns = ['sa2_code', 'geometry']

In [ ]:
gdf_sa2['sa2_code'] = gdf_sa2['sa2_code'].astype(str)
demographics['sa2_code'] = demographics['sa2_code'].astype(str)
gdf = gdf_sa2.merge(demographics, on='sa2_code', how='inner')
gdf = gdf.replace([np.inf, -np.inf], np.nan).dropna()
gdf = gdf.to_crs(epsg=4326)
gdf

In [ ]:
m = folium.Map(location=[-37.81, 144.96], zoom_start=10, tiles="cartodbpositron")

In [ ]:
# -------------------------------
# Choose a Variable to Visualize
# -------------------------------
# metric = 'rent_to_income_ratio'
metric = 'pct_students'
tooltip_fields = ['sa2_code', 'median_weekly_rent', 'median_weekly_income', 'rent_to_income_ratio']

# Add Choropleth Layer
folium.Choropleth(
    geo_data=gdf,
    name='Rent to Income Ratio',
    data=gdf,
    columns=['sa2_code', metric],
    key_on='feature.properties.sa2_code',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='gray',
    legend_name='Rent to Income Ratio'
).add_to(m)

# Add Interactive Tooltip
folium.GeoJson(
    gdf,
    name='Suburb Info',
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
    tooltip=GeoJsonTooltip(fields=tooltip_fields)
).add_to(m)

# Show Map
m